In [5]:
# ============================================================
# Smart Recycling Bin - Trash Classification with MobileNetV2
# ============================================================

# 1️⃣ Import Libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.preprocessing import image_dataset_from_directory
# Install matplotlib if not already installed
# %pip install matplotlib

import matplotlib.pyplot as plt
import numpy as np
import os

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.13.0


In [ ]:
# ------------------------------------------------------------
# Ganti path dataset sesuai lokasi kamu
# Folder struktur:
# dataset/
#   ├── cardboard/
#   ├── glass/
#   ├── metal/
#   ├── paper/
#   ├── plastic/
#   └── trash/
# ------------------------------------------------------------

DATA_DIR = "dataset"
IMG_SIZE = 160
BATCH_SIZE = 32

train_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print("Classes:", class_names)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [ ]:
# TODO: Gunakan MobileNetV2 pretrained ImageNet
base_model = mobilenet_v2.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze backbone

# Bangun model
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = models.Model(inputs, outputs)

model.summary()


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# TODO: Plot hasil training
plt.plot(history_head.history['accuracy'], label='train_acc')
plt.plot(history_head.history['val_accuracy'], label='val_acc')
plt.legend()
plt.title("Training Head Only")
plt.show()


In [ ]:
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# TODO: Plot hasil fine-tuning
plt.plot(history_ft.history['accuracy'], label='train_acc')
plt.plot(history_ft.history['val_accuracy'], label='val_acc')
plt.legend()
plt.title("Fine-Tuning")
plt.show()


In [ ]:
model.save("waste_classifier_v1.keras")
print("✅ Model saved to waste_classifier_v1.keras")

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array

# TODO: ganti dengan path gambar uji kamu
test_img_path = "sample_test/plastic_bottle.jpg"

img = load_img(test_img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = mobilenet_v2.preprocess_input(img_array)

pred = model.predict(img_array)
pred_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print(f"Predicted Class: {pred_class}")
print(f"Confidence: {confidence:.2f}")

plt.imshow(load_img(test_img_path))
plt.title(f"Predicted: {pred_class} ({confidence:.2f})")
plt.axis("off")
plt.show()


In [ ]:
# TODO: Evaluasi pada validation/test set
loss, acc = model.evaluate(val_ds)
print(f"Validation accuracy: {acc:.4f}")

# TODO: Buat confusion matrix (opsional)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = np.concatenate([y for x, y in val_ds], axis=0)
y_pred = np.argmax(model.predict(val_ds), axis=1)

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(xticks_rotation=45)
plt.title("Confusion Matrix")
plt.show()